# 🎨 ComfyUI Model Download Manager

**Interactive model downloader with storage management for Paperspace**

---

## 🔍 1. Check Storage Status

In [ ]:
import os
import shutil
import subprocess
from pathlib import Path

# Storage paths
STORAGE_PATH = Path('/storage/ComfyUI')
TEMP_STORAGE = Path('/temp-storage/ComfyUI')
PROJECT_PATH = Path('/notebooks/comfyui-paperspace-notebook/ComfyUI')

def check_storage():
    print("📊 STORAGE STATUS")
    print("================")
    
    for name, path in [("Permanent", "/storage"), ("Temporary", "/temp-storage"), ("Project", "/notebooks")]:
        try:
            if os.path.exists(path):
                stat = shutil.disk_usage(path)
                total_gb = stat.total / (1024**3)
                used_gb = stat.used / (1024**3)
                free_gb = stat.free / (1024**3)
                percent = (stat.used / stat.total) * 100
                
                # Color coding
                if percent > 90:
                    status = "🔴"
                elif percent > 70:
                    status = "🟡"
                else:
                    status = "🟢"
                
                print(f"\n{status} {name} Storage ({path}):")
                print(f"   Total: {total_gb:.1f}GB | Used: {used_gb:.1f}GB | Free: {free_gb:.1f}GB")
                print(f"   Usage: {percent:.1f}%")
            else:
                print(f"\n⚠️ {name} Storage: Not available")
        except Exception as e:
            print(f"\n❌ {name} Storage: Error - {e}")

check_storage()

## 🎯 2. Quick Model Download

In [ ]:
# Quick download for essential models
import subprocess
import os
from pathlib import Path

def download_model(url, dest_path, filename=None):
    """Download a model with progress bar"""
    if not filename:
        filename = os.path.basename(url)
    
    # Create destination directory
    Path(dest_path).mkdir(parents=True, exist_ok=True)
    
    file_path = Path(dest_path) / filename
    
    # Skip if exists
    if file_path.exists():
        print(f"✅ Already exists: {filename}")
        return True
    
    print(f"⬇️ Downloading: {filename}")
    print(f"   To: {file_path}")
    
    # Download with wget (shows progress)
    cmd = f"wget -c -O '{file_path}' '{url}'"
    result = subprocess.run(cmd, shell=True)
    
    if result.returncode == 0:
        print(f"✅ Downloaded: {filename}")
        return True
    else:
        print(f"❌ Failed: {filename}")
        return False

# Essential models
print("📦 Essential Models for Quick Start:")
print("====================================")
print("")
print("Choose models to download:")
print("1. SDXL Base 1.0 (6.9GB) - Recommended")
print("2. SDXL VAE (335MB) - Recommended")
print("3. SD 1.5 (2GB) - Smaller, faster")
print("4. ControlNet (1.4GB)")
print("5. All essential models")
print("0. Skip")

choice = input("\nEnter choice (0-5): ")

# Use permanent storage if available, else temp
base_path = Path('/storage/models') if os.path.exists('/storage') else Path('/temp-storage/models')

if choice == "1" or choice == "5":
    download_model(
        "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors",
        base_path / "checkpoints"
    )

if choice == "2" or choice == "5":
    download_model(
        "https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors",
        base_path / "vae"
    )

if choice == "3":
    download_model(
        "https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors",
        base_path / "checkpoints"
    )

if choice == "4":
    download_model(
        "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth",
        base_path / "controlnet"
    )

if choice != "0":
    print("\n✅ Download complete!")
    print(f"Models saved to: {base_path}")

## 🎨 3. Chroma Models

In [ ]:
# Download Chroma models
print("🎨 CHROMA MODEL DOWNLOADER")
print("==========================")
print("")
print("Available Chroma Models:")
print("1. Chroma v48 Latest (24GB)")
print("2. Chroma v48 Detail (24GB)")
print("3. Chroma v29.5 (24GB)")
print("4. Chroma VAE (335MB) - Required")
print("5. Text Encoder FP8 (4.9GB) - Required")
print("0. Skip")

chroma_choice = input("\nSelect models (comma-separated, e.g., 1,4,5): ")

if chroma_choice and chroma_choice != "0":
    base_path = Path('/storage/models') if os.path.exists('/storage') else Path('/temp-storage/models')
    choices = [c.strip() for c in chroma_choice.split(',')]
    
    chroma_models = {
        "1": ("https://huggingface.co/lodestones/Chroma/resolve/main/chroma-unlocked-v48.safetensors", "unet/Chroma"),
        "2": ("https://huggingface.co/lodestones/Chroma/resolve/main/chroma-unlocked-v48-detail-calibrated.safetensors", "unet/Chroma"),
        "3": ("https://huggingface.co/lodestones/Chroma/resolve/main/chroma-unlocked-v29.5.safetensors", "unet/Chroma"),
        "4": ("https://huggingface.co/lodestones/Chroma/resolve/main/ae.safetensors", "vae/Chroma"),
        "5": ("https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp8_e4m3fn_scaled.safetensors", "text_encoders/Chroma")
    }
    
    for choice in choices:
        if choice in chroma_models:
            url, subdir = chroma_models[choice]
            download_model(url, base_path / subdir)
    
    print("\n✅ Chroma models downloaded!")

## 🔗 4. Create Symlinks

In [ ]:
# Create symlinks from storage to ComfyUI
import os
from pathlib import Path

def create_symlinks():
    print("🔗 Creating symlinks to ComfyUI...")
    print("===================================")
    
    # Source paths (where models are stored)
    storage_paths = [
        Path('/storage/models'),
        Path('/temp-storage/models')
    ]
    
    # Target path (ComfyUI models directory)
    comfyui_models = Path('/notebooks/comfyui-paperspace-notebook/ComfyUI/models')
    
    if not comfyui_models.exists():
        print(f"❌ ComfyUI models directory not found: {comfyui_models}")
        return
    
    # Find which storage path has models
    source_path = None
    for path in storage_paths:
        if path.exists():
            source_path = path
            break
    
    if not source_path:
        print("❌ No model storage found")
        return
    
    print(f"Source: {source_path}")
    print(f"Target: {comfyui_models}")
    print("")
    
    # Create symlinks for each subdirectory
    subdirs = ['checkpoints', 'vae', 'loras', 'controlnet', 'unet', 'text_encoders', 'clip']
    
    for subdir in subdirs:
        source = source_path / subdir
        target = comfyui_models / subdir
        
        if source.exists():
            # Remove existing symlink or directory
            if target.is_symlink():
                target.unlink()
            elif target.exists():
                print(f"⚠️ {subdir} exists and is not a symlink, skipping")
                continue
            
            # Create symlink
            try:
                os.symlink(source, target)
                print(f"✅ Linked: {subdir}")
            except Exception as e:
                print(f"❌ Failed to link {subdir}: {e}")
        else:
            print(f"⚠️ Source not found: {subdir}")
    
    print("\n✅ Symlinks created!")

create_symlinks()

## 📝 5. List Downloaded Models

In [ ]:
# List all downloaded models
from pathlib import Path

def list_models():
    print("📦 DOWNLOADED MODELS")
    print("===================")
    
    paths = [
        ("Permanent", Path('/storage/models')),
        ("Temporary", Path('/temp-storage/models')),
        ("ComfyUI", Path('/notebooks/comfyui-paperspace-notebook/ComfyUI/models'))
    ]
    
    for name, path in paths:
        if path.exists():
            print(f"\n📁 {name} Storage:")
            
            # Find all model files
            models = list(path.rglob('*.safetensors')) + \
                    list(path.rglob('*.ckpt')) + \
                    list(path.rglob('*.pth')) + \
                    list(path.rglob('*.gguf'))
            
            if models:
                for model in sorted(models):
                    size_mb = model.stat().st_size / (1024**2)
                    rel_path = model.relative_to(path)
                    
                    if size_mb > 1024:
                        size_str = f"{size_mb/1024:.1f}GB"
                    else:
                        size_str = f"{size_mb:.1f}MB"
                    
                    print(f"   • {rel_path} ({size_str})")
            else:
                print(f"   No models found")
        else:
            print(f"\n⚠️ {name} Storage: Not found")

list_models()

## 🗑️ 6. Storage Cleanup

In [ ]:
# Cleanup temporary storage
import shutil
from pathlib import Path

def cleanup_storage():
    print("🗑️ STORAGE CLEANUP")
    print("==================")
    print("")
    print("WARNING: This will delete files!")
    print("")
    print("Options:")
    print("1. Clear temporary storage (/temp-storage)")
    print("2. Clear pip cache")
    print("3. Clear ComfyUI outputs")
    print("4. Clear old logs")
    print("0. Cancel")
    
    choice = input("\nEnter choice: ")
    
    if choice == "1":
        temp_path = Path('/temp-storage')
        if temp_path.exists():
            size = sum(f.stat().st_size for f in temp_path.rglob('*') if f.is_file()) / (1024**3)
            print(f"\nThis will free {size:.2f}GB")
            confirm = input("Are you sure? (yes/no): ")
            if confirm.lower() == 'yes':
                shutil.rmtree(temp_path)
                temp_path.mkdir()
                print("✅ Temporary storage cleared")
        else:
            print("No temporary storage found")
    
    elif choice == "2":
        subprocess.run("pip cache purge", shell=True)
        print("✅ Pip cache cleared")
    
    elif choice == "3":
        output_path = Path('/notebooks/comfyui-paperspace-notebook/ComfyUI/output')
        if output_path.exists():
            for f in output_path.glob('*'):
                if f.is_file():
                    f.unlink()
            print("✅ Outputs cleared")
    
    elif choice == "4":
        log_paths = [
            Path('/notebooks/comfyui-paperspace-notebook/logs'),
            Path('/tmp'),
            Path('/var/log')
        ]
        for path in log_paths:
            if path.exists():
                for f in path.glob('*.log'):
                    try:
                        f.unlink()
                    except:
                        pass
        print("✅ Old logs cleared")

cleanup_storage()

---

## 🎉 **Model Management Complete!**

**Your models are ready to use in ComfyUI**

### 💡 **Tips:**

- **Permanent Storage**: Use `/storage` for models (50GB limit, survives restarts)
- **Temporary Storage**: Use `/temp-storage` for testing (faster, lost on restart)
- **Symlinks**: Automatically link models to ComfyUI
- **Space Management**: Monitor your 50GB limit carefully

### 🚀 **Next Steps:**

1. Run ComfyUI: `cd /notebooks/comfyui-paperspace-notebook && ./run.sh`
2. Access at: `https://[YOUR-ID]-8188.paperspacegradient.com`
3. Load your downloaded models in ComfyUI

---